## Intalling the necessary dependencies

In [2]:
!pip install fastapi
!pip install uvicorn
!pip install pypi-json
!pip install requests
!pip install pydantic
!pip install pickle5
!pip install scikit-learn
!pip install pyngrok
!pip install nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 14.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simp

## Importing all the needed libraries

In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import requests
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

## Creating an instance of FastAPI

In [4]:
app = FastAPI()

## Configuring Cross-Origin Resource Sharing (CORS)

In [5]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

## Defining a schemae for a machine learning model input

In [6]:
class model_input(BaseModel):
  Pregnancies : int
  Glucose : int
  BloodPressure : int
  SkinThickness : int
  Insulin : int
  BMI : float
  DiabetesPedigreeFunction : float
  Age : int

## Loading the trained (saved) model

In [7]:
diabetes_disease_model = pickle.load(open('diabetes_disease_model.sav','rb'))

## Defining a diabetes prediction function

In [8]:
@app.post('/diabetes_prediction')
def diabetes_prediction(input_parameters: model_input):
  input_data = input_parameters.json()
  input_dict = json.loads(input_data)

  pre = input_dict['Pregnancies']
  glu = input_dict['Glucose']
  bp = input_dict['BloodPressure']
  st = input_dict['SkinThickness']
  ins = input_dict['Insulin']
  bmi = input_dict['BMI']
  dpf = input_dict['DiabetesPedigreeFunction']
  age = input_dict['Age']

  prediction = diabetes_disease_model.predict([[pre,glu,bp,st,ins,bmi,dpf,age]])
  if prediction[0] == 0:
    return 'This person is not diabetic!'
  else:
    return 'This person is diabetic!'

## Exposing locally running FastAPI application to the internet via temperary public URL

In [10]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL: ', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL:  https://57ea-34-80-222-190.ngrok.io


INFO:     Started server process [2137]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     105.67.131.172:0 - "POST /diabetes_prediction HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2137]
